In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
# Load your raw data
df = pd.read_csv('../data/raw/data.csv')




In [34]:
# Define your categorical and numerical columns
categorical_cols = ['ProviderId', 'ProductCategory', 'ChannelId', 'ProductId']
numerical_cols = ['Amount', 'Value']

In [35]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Categorical pipeline
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Numerical pipeline
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Combine into a ColumnTransformer
preprocessor = ColumnTransformer([
    ('cat', categorical_pipeline, categorical_cols),
    ('num', numerical_pipeline, numerical_cols)
])

In [36]:
# Only transform the feature columns, not CustomerId
X = df[categorical_cols + numerical_cols]
X_processed = preprocessor.fit_transform(X)
print(X_processed.shape)  # Should be (n_rows, n_features)

(95662, 44)


In [37]:
# Get feature names for the one-hot encoded columns
cat_features = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_cols)
feature_names = list(cat_features) + numerical_cols
print(len(feature_names))  # Should match X_processed.shape[1]
print(feature_names)

44
['ProviderId_ProviderId_1', 'ProviderId_ProviderId_2', 'ProviderId_ProviderId_3', 'ProviderId_ProviderId_4', 'ProviderId_ProviderId_5', 'ProviderId_ProviderId_6', 'ProductCategory_airtime', 'ProductCategory_data_bundles', 'ProductCategory_financial_services', 'ProductCategory_movies', 'ProductCategory_other', 'ProductCategory_ticket', 'ProductCategory_transport', 'ProductCategory_tv', 'ProductCategory_utility_bill', 'ChannelId_ChannelId_1', 'ChannelId_ChannelId_2', 'ChannelId_ChannelId_3', 'ChannelId_ChannelId_5', 'ProductId_ProductId_1', 'ProductId_ProductId_10', 'ProductId_ProductId_11', 'ProductId_ProductId_12', 'ProductId_ProductId_13', 'ProductId_ProductId_14', 'ProductId_ProductId_15', 'ProductId_ProductId_16', 'ProductId_ProductId_19', 'ProductId_ProductId_2', 'ProductId_ProductId_20', 'ProductId_ProductId_21', 'ProductId_ProductId_22', 'ProductId_ProductId_23', 'ProductId_ProductId_24', 'ProductId_ProductId_27', 'ProductId_ProductId_3', 'ProductId_ProductId_4', 'ProductId_Pr

In [ ]:
# Create DataFrame with processed features
processed_df = pd.DataFrame(X_processed, columns=feature_names)

# Add CustomerId as a column
processed_df['CustomerId'] = df['CustomerId'].values

# Save to CSV
processed_df.to_csv('../data/processed/processedData.csv', index=False)
print(processed_df.head())

In [39]:
# Select only the RFM columns for scaling
rfm_features = rfm[['Recency', 'Frequency', 'Monetary']]

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the RFM features
rfm_scaled = scaler.fit_transform(rfm_features)

# Optional: Convert back to DataFrame for easier inspection
rfm_scaled_df = pd.DataFrame(rfm_scaled, columns=['Recency', 'Frequency', 'Monetary'])
print(rfm_scaled_df.head())

    Recency  Frequency  Monetary
0  1.937605  -0.253459 -0.066891
1  1.937605  -0.253459 -0.066891
2  2.158882  -0.212186 -0.055849
3 -0.201408  -0.150278 -0.061655
4 -0.717722  -0.201868 -0.055849


In [40]:
# Ensure TransactionStartTime is datetime
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

# Define the snapshot date
snapshot_date = df['TransactionStartTime'].max() + pd.Timedelta(days=1)

# Group by CustomerId to calculate RFM
rfm = df.groupby('CustomerId').agg({
    'TransactionStartTime': lambda x: (snapshot_date - x.max()).days,  # Recency
    'CustomerId': 'count',                                            # Frequency
    'Amount': 'sum'                                                   # Monetary
}).rename(columns={
    'TransactionStartTime': 'Recency',
    'CustomerId': 'Frequency',
    'Amount': 'Monetary'
}).reset_index()

# Display the first few rows
print(rfm.head())

        CustomerId  Recency  Frequency  Monetary
0     CustomerId_1       84          1  -10000.0
1    CustomerId_10       84          1  -10000.0
2  CustomerId_1001       90          5   20000.0
3  CustomerId_1002       26         11    4225.0
4  CustomerId_1003       12          6   20000.0


In [41]:
# Set the number of clusters and random_state for reproducibility
n_clusters = 3
random_state = 42

# Initialize and fit KMeans
kmeans = KMeans(n_clusters=n_clusters, random_state=random_state)
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)

# Inspect the cluster assignments
print(rfm.groupby('Cluster')[['Recency', 'Frequency', 'Monetary']].mean())
print(rfm.head())

           Recency    Frequency      Monetary
Cluster                                      
0        61.859846     7.726699  8.172379e+04
1        29.000000  4091.000000 -1.049000e+08
2        12.716076    34.807692  2.726546e+05
        CustomerId  Recency  Frequency  Monetary  Cluster
0     CustomerId_1       84          1  -10000.0        0
1    CustomerId_10       84          1  -10000.0        0
2  CustomerId_1001       90          5   20000.0        0
3  CustomerId_1002       26         11    4225.0        2
4  CustomerId_1003       12          6   20000.0        2


In [42]:
# Assign high risk to customers in Cluster 0
high_risk_cluster = 0  # Based on your analysis above

rfm['is_high_risk'] = (rfm['Cluster'] == high_risk_cluster).astype(int)

# Check the result
print(rfm[['CustomerId', 'Cluster', 'is_high_risk']].head())

        CustomerId  Cluster  is_high_risk
0     CustomerId_1        0             1
1    CustomerId_10        0             1
2  CustomerId_1001        0             1
3  CustomerId_1002        2             0
4  CustomerId_1003        2             0


In [43]:

merged_df = processed_df.merge(rfm[['CustomerId', 'is_high_risk']], on='CustomerId', how='left')
print(merged_df.head())

                                                   0       CustomerId  \
0    (0, 5)\t1.0\n  (0, 6)\t1.0\n  (0, 17)\t1.0\n...  CustomerId_4406   
1    (0, 3)\t1.0\n  (0, 8)\t1.0\n  (0, 16)\t1.0\n...  CustomerId_4406   
2    (0, 5)\t1.0\n  (0, 6)\t1.0\n  (0, 17)\t1.0\n...  CustomerId_4683   
3    (0, 0)\t1.0\n  (0, 14)\t1.0\n  (0, 17)\t1.0\...   CustomerId_988   
4    (0, 3)\t1.0\n  (0, 8)\t1.0\n  (0, 16)\t1.0\n...   CustomerId_988   

   is_high_risk  
0             0  
1             0  
2             1  
3             0  
4             0  


In [44]:
import numpy as np

# If X_processed is a sparse matrix, convert it to a dense array
if hasattr(X_processed, "toarray"):
    X_processed = X_processed.toarray()

In [46]:
processed_df = pd.DataFrame(X_processed, columns=feature_names)
processed_df['CustomerId'] = df['CustomerId'].values
processed_df.to_csv('../data/processed/processed_data.csv', index=False)
print(processed_df.head())

   ProviderId_ProviderId_1  ProviderId_ProviderId_2  ProviderId_ProviderId_3  \
0                      0.0                      0.0                      0.0   
1                      0.0                      0.0                      0.0   
2                      0.0                      0.0                      0.0   
3                      1.0                      0.0                      0.0   
4                      0.0                      0.0                      0.0   

   ProviderId_ProviderId_4  ProviderId_ProviderId_5  ProviderId_ProviderId_6  \
0                      0.0                      0.0                      1.0   
1                      1.0                      0.0                      0.0   
2                      0.0                      0.0                      1.0   
3                      0.0                      0.0                      0.0   
4                      1.0                      0.0                      0.0   

   ProductCategory_airtime  ProductCat

In [48]:
   X_processed = preprocessor.fit_transform(X)
   if hasattr(X_processed, "toarray"):
       X_processed = X_processed.toarray()

In [49]:
processed_df = pd.DataFrame(X_processed, columns=feature_names)

In [50]:
processed_df['CustomerId'] = df['CustomerId'].values

In [51]:
processed_df.to_csv('../data/processed/processedData.csv', index=False)

In [52]:
print(processed_df.head())

   ProviderId_ProviderId_1  ProviderId_ProviderId_2  ProviderId_ProviderId_3  \
0                      0.0                      0.0                      0.0   
1                      0.0                      0.0                      0.0   
2                      0.0                      0.0                      0.0   
3                      1.0                      0.0                      0.0   
4                      0.0                      0.0                      0.0   

   ProviderId_ProviderId_4  ProviderId_ProviderId_5  ProviderId_ProviderId_6  \
0                      0.0                      0.0                      1.0   
1                      1.0                      0.0                      0.0   
2                      0.0                      0.0                      1.0   
3                      0.0                      0.0                      0.0   
4                      1.0                      0.0                      0.0   

   ProductCategory_airtime  ProductCat

In [53]:
# Assuming rfm has columns 'CustomerId' and 'is_high_risk'
merged_df = processed_df.merge(rfm[['CustomerId', 'is_high_risk']], on='CustomerId', how='left')
print(merged_df.head())

   ProviderId_ProviderId_1  ProviderId_ProviderId_2  ProviderId_ProviderId_3  \
0                      0.0                      0.0                      0.0   
1                      0.0                      0.0                      0.0   
2                      0.0                      0.0                      0.0   
3                      1.0                      0.0                      0.0   
4                      0.0                      0.0                      0.0   

   ProviderId_ProviderId_4  ProviderId_ProviderId_5  ProviderId_ProviderId_6  \
0                      0.0                      0.0                      1.0   
1                      1.0                      0.0                      0.0   
2                      0.0                      0.0                      1.0   
3                      0.0                      0.0                      0.0   
4                      1.0                      0.0                      0.0   

   ProductCategory_airtime  ProductCat